In [2]:
import pandas as pd
from pathlib import Path
#import pyarrow.parquet as pq

train_path = Path(Path.cwd(), 'Data', 'train_series.parquet')
train_path

WindowsPath('c:/Users/Sol/Desktop/IMT/AML/Code/Project_Sleep_States/Data/train_series.parquet')

In [4]:
# table = pq.read_table(path)
# train_series = table.to_pandas()  # Convert to Pandas DataFrame if needed
train_series = pd.read_parquet(train_path)
print(train_series.head())

      series_id  step                 timestamp  anglez    enmo
0  038441c925bb     0  2018-08-14T15:30:00-0400  2.6367  0.0217
1  038441c925bb     1  2018-08-14T15:30:05-0400  2.6368  0.0215
2  038441c925bb     2  2018-08-14T15:30:10-0400  2.6370  0.0216
3  038441c925bb     3  2018-08-14T15:30:15-0400  2.6368  0.0213
4  038441c925bb     4  2018-08-14T15:30:20-0400  2.6368  0.0215


In [5]:
train_path = Path(Path.cwd(), 'Data', 'train_events.csv')
train_events = pd.read_csv(train_path)

In [6]:
print(train_series.head())
print(train_events.head())

print(train_series.info())
print(train_events.info())

      series_id  step                 timestamp  anglez    enmo
0  038441c925bb     0  2018-08-14T15:30:00-0400  2.6367  0.0217
1  038441c925bb     1  2018-08-14T15:30:05-0400  2.6368  0.0215
2  038441c925bb     2  2018-08-14T15:30:10-0400  2.6370  0.0216
3  038441c925bb     3  2018-08-14T15:30:15-0400  2.6368  0.0213
4  038441c925bb     4  2018-08-14T15:30:20-0400  2.6368  0.0215
      series_id  night   event     step                 timestamp
0  038441c925bb      1   onset   4992.0  2018-08-14T22:26:00-0400
1  038441c925bb      1  wakeup  10932.0  2018-08-15T06:41:00-0400
2  038441c925bb      2   onset  20244.0  2018-08-15T19:37:00-0400
3  038441c925bb      2  wakeup  27492.0  2018-08-16T05:41:00-0400
4  038441c925bb      3   onset  39996.0  2018-08-16T23:03:00-0400
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   tim

In [10]:
train_series = train_series[:10000]
train_events = train_events[:10000]

In [15]:
df = train_series.merge(train_events, on="series_id", how="left")

In [16]:
df

,series_id,step_x,timestamp_x,anglez,enmo,night,event,step_y,timestamp_y
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,0,2018-08-14T15:30:00-0400,2.6367,0.0217,3,onset,39996.0,2018-08-16T23:03:00-0400
...,...,...,...,...,...,...,...,...,...
459995,038441c925bb,9999,2018-08-15T05:23:15-0400,-13.0864,0.0085,21,wakeup,355944.0,2018-09-04T05:52:00-0400
459996,038441c925bb,9999,2018-08-15T05:23:15-0400,-13.0864,0.0085,22,onset,367020.0,2018-09-04T21:15:00-0400
459997,038441c925bb,9999,2018-08-15T05:23:15-0400,-13.0864,0.0085,22,wakeup,369912.0,2018-09-05T01:16:00-0400
459998,038441c925bb,9999,2018-08-15T05:23:15-0400,-13.0864,0.0085,23,onset,NaN,NaN


In [17]:
df = df.sort_values(by=["series_id", "step_x"])

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Crear etiquetas (1 si es onset/wakeup, 0 si no)
df['target'] = df['event'].notnull().astype(int)

# Seleccionar variables de entrada
features = ["anglez", "enmo"]
X = df[features]
y = df["target"]

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluar
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     92000

    accuracy                           1.00     92000
   macro avg       1.00      1.00      1.00     92000
weighted avg       1.00      1.00      1.00     92000



In [19]:
test_path = Path(Path.cwd(), 'Data', 'test_series.parquet')
test_series = pd.read_parquet(test_path)
print(test_series.head())

      series_id  step                 timestamp  anglez    enmo
0  038441c925bb     0  2018-08-14T15:30:00-0400  2.6367  0.0217
1  038441c925bb     1  2018-08-14T15:30:05-0400  2.6368  0.0215
2  038441c925bb     2  2018-08-14T15:30:10-0400  2.6370  0.0216
3  038441c925bb     3  2018-08-14T15:30:15-0400  2.6368  0.0213
4  038441c925bb     4  2018-08-14T15:30:20-0400  2.6368  0.0215


In [21]:
# Seleccionar las mismas características que usaste para entrenar
X_test = test_series[["anglez", "enmo"]]
X_test

,anglez,enmo
0,2.636700,0.0217
1,2.636800,0.0215
2,2.637000,0.0216
3,2.636800,0.0213
4,2.636800,0.0215
...,...,...
445,-59.696899,0.0601
446,-35.656601,0.0427
447,-21.582399,0.0309
448,-42.616001,0.0328


In [23]:
test_series["predictions"] = clf.predict(X_test)
test_series["score"] = clf.predict_proba(X_test)[:, 0]

events_detected = test_series[test_series["predictions"] == 1].copy()

In [24]:
print(pd.Series(y_train).value_counts())

target
1    368000
Name: count, dtype: int64


In [ ]:
import numpy as np
events_detected["event"] = np.where(np.random.rand(len(events_detected)) > 0.5, "onset", "wakeup")

# Crear un ID único para cada evento detectado
events_detected["row_id"] = range(len(events_detected))

# Seleccionar columnas requeridas
submission = events_detected[["row_id", "series_id", "step", "event", "score"]]

# Guardar archivo CSV
submission.to_csv("submission.csv", index=False)

print("Archivo submission.csv guardado.")